In [1]:
import numpy as np
import os
import torch
from misc.example_helper import *
import importlib
import processing
import models
%load_ext autoreload
%autoreload 2
importlib.reload(processing)
importlib.reload(models)
from processing.data_manager import SHREDDataManager
from processing.parametric_data_manager import ParametricSHREDDataManager
from models.shred_models import SHRED
from models.shred_models import evaluate
import os


In [51]:
# Initialize SHREDDataManager
manager = SHREDDataManager(
    lags = 20,
    train_size = 0.6,
    val_size = 0.2,
    test_size = 0.2,
    scaling=True,
    # compression = 50,
    compression=False,
    time=np.arange(0, 2000),
    method="all"
)

input_folder = "data4fields_npz"
filename = 'ni_3D.npz'

file_path = os.path.join(input_folder, filename)
ni_3D = np.load(file_path)
ni_3D = ni_3D[ni_3D.files[0]]
mobile_sensors = [
    perimeter_walk(height = ni_3D.shape[1], width = ni_3D.shape[2], timesteps = ni_3D.shape[0], clockwise=True),
    perimeter_walk(height = ni_3D.shape[1], width = ni_3D.shape[2], timesteps = ni_3D.shape[0], clockwise=False)
]
manager.add_field(
    data=ni_3D,
    random_sensors=3,
    # stationary_sensors=[(7, 9), (0,0)],
    # mobile_sensors=mobile_sensors,
    id = str(os.path.splitext(filename)[0])
)

hello scaler
hello scaler


In [52]:
manager.input_summary

,field id,sensor id,sensor type,location/trajectory
0,ni_3D,ni_3D-0,stationary (randomly selected),"(239, 159)"
1,ni_3D,ni_3D-1,stationary (randomly selected),"(252, 14)"
2,ni_3D,ni_3D-2,stationary (randomly selected),"(44, 150)"


In [59]:
manager.sensor_measurements # always shows unscaled, raw sensor data

,time,ni_3D-0,ni_3D-1,ni_3D-2
0,0,4.666700e+16,4.998900e+16,5.113500e+16
1,1,4.572000e+16,4.863200e+16,5.178500e+16
2,2,4.580800e+16,4.363800e+16,5.334600e+16
3,3,4.717700e+16,3.948800e+16,5.442900e+16
4,4,4.783800e+16,3.754100e+16,5.590800e+16
...,...,...,...,...
1995,1995,6.343200e+16,4.071800e+16,7.914700e+16
1996,1996,5.914700e+16,3.822000e+16,8.671300e+16
1997,1997,5.805700e+16,3.874700e+16,9.221100e+16
1998,1998,6.124500e+16,4.107300e+16,1.020900e+17


In [54]:
train_set, valid_set, test_set = manager.preprocess()

In [ ]:
ni_3D_train_DM = train_set.predictor_dataset.Y.numpy()
ni_3D_train_DM.shape

(1200, 65792)

In [56]:
ni_3D_train = ni_3D[0:1200,:]
ni_3D_train = ni_3D_train.reshape(ni_3D_train.shape[0], -1)
ni_3D_train.shape

(1200, 65792)

In [57]:
ni_3D_train_DM

array([[1.        , 0.95407444, 1.        , ..., 1.        , 0.9563991 ,
        1.        ],
       [0.9243055 , 0.824495  , 0.92970234, ..., 0.8924829 , 0.9692821 ,
        0.97635794],
       [0.8619713 , 0.6620216 , 0.6703959 , ..., 0.78196603, 0.7695233 ,
        0.6334361 ],
       ...,
       [0.47731158, 0.4794964 , 0.46769837, ..., 0.53957886, 0.5741205 ,
        0.7164786 ],
       [0.5571379 , 0.39868003, 0.4204685 , ..., 0.6079217 , 0.680575  ,
        0.66695446],
       [0.5705402 , 0.47571304, 0.44512182, ..., 0.6611869 , 0.68104756,
        0.5992457 ]], shape=(1200, 65792), dtype=float32)

In [28]:
ni_3D_train

array([[5.6167e+16, 5.5958e+16, 5.5041e+16, ..., 5.6783e+16, 5.2811e+16,
        5.4436e+16],
       [5.2558e+16, 4.9793e+16, 5.1893e+16, ..., 5.1335e+16, 5.3438e+16,
        5.3275e+16],
       [4.9586e+16, 4.2063e+16, 4.0281e+16, ..., 4.5735e+16, 4.3716e+16,
        3.6435e+16],
       ...,
       [3.1246e+16, 3.3379e+16, 3.1204e+16, ..., 3.3453e+16, 3.4206e+16,
        4.0513e+16],
       [3.5052e+16, 2.9534e+16, 2.9089e+16, ..., 3.6916e+16, 3.9387e+16,
        3.8081e+16],
       [3.5691e+16, 3.3199e+16, 3.0193e+16, ..., 3.9615e+16, 3.9410e+16,
        3.4756e+16]], shape=(1200, 65792))

In [ ]:
unscaled = manager.postprocess(data = ni_3D_train_DM, uncompress = False, unscale = True, method = 'predictor')

method temporal_reconstructor
(1200, 65792)


In [61]:
unscaled

{'ni_3D': array([[5.6167001e+16, 5.5957999e+16, 5.5040998e+16, ..., 5.6782998e+16,
         5.2810999e+16, 5.4435996e+16],
        [5.2558000e+16, 4.9792999e+16, 5.1893002e+16, ..., 5.1334995e+16,
         5.3437996e+16, 5.3274998e+16],
        [4.9586003e+16, 4.2062998e+16, 4.0280999e+16, ..., 4.5735002e+16,
         4.3716002e+16, 3.6435002e+16],
        ...,
        [3.1246001e+16, 3.3379000e+16, 3.1204000e+16, ..., 3.3453000e+16,
         3.4206000e+16, 4.0513001e+16],
        [3.5051999e+16, 2.9533999e+16, 2.9089000e+16, ..., 3.6916000e+16,
         3.9387002e+16, 3.8081001e+16],
        [3.5691000e+16, 3.3199000e+16, 3.0192999e+16, ..., 3.9615000e+16,
         3.9410001e+16, 3.4756001e+16]], shape=(1200, 65792), dtype=float32)}

In [63]:
# initialize SHRED
shred = SHRED(sequence='LSTM', decoder='SDN')
# fit SHRED
valid_errors = shred.fit(train_set, valid_set, num_epochs=50, patience=20, verbose=True)

Newest Version
input_size 3
output_size 65792
self._sequence_model_random_reconstructor.output_size 64

Fitting Random Reconstructor...










































Epoch 1/50: 100%|██████████| 19/19 [00:03<00:00,  5.68batch/s, loss=0.12, L2=0.619, val_loss=0.0295, val_L2=0.334]








































Epoch 2/50: 100%|██████████| 19/19 [00:03<00:00,  5.76batch/s, loss=0.0296, L2=0.332, val_loss=0.0259, val_L2=0.312]





KeyboardInterrupt: 